In [ ]:
# Import used packages
%matplotlib notebook
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
import random
from matplotlib import animation
import time

In [ ]:
# Create positions in rectangle of xlen x ylen
def randompos(nodes, xlen, ylen):
    pos = []
    for i in range(nodes):
        pos.append([xlen*random.uniform(0, 1), ylen*random.uniform(0, 1)])
    return pos

# Link nodes
def linknodes(pos, radius):
    links = []
    sets = []
    for i in range(len(pos)):
        linksCurrent = []
        for j in range(len(pos)):
            distance = (pos[i][0] - pos[j][0])**2 + (pos[i][1] - pos[j][1])**2
            if distance <= radius**2:
                if i != j:
                    linksCurrent.append(j)
                    if (i, j) not in sets:
                        sets.append((i, j))
        links.append(linksCurrent)
    return links, sets
    

# Create network in a dictionary structure
def createnetwork(nodes, radius, xlen, ylen):
    Network = {'nodes': nodes, 'pos': randompos(nodes, pictureWidth, pictureHeight)}
    Network['links'], Network['sets'] = linknodes(Network['pos'], radius)
    return Network

# plot nodes
def plotnodes(pos, ax, sizeNodes, colorNodes):
    ax.scatter([item[0] for item in pos], [item[1] for item in pos], s= sizeNodes, c= colorNodes, zorder= 2)
    
#plot edges
def plotedges(pos, ax, sets, colorEdges, alphaEdges):
    xList = []
    yList = []
    for linkedNodes in sets:
        for element in linkedNodes:
            xList += [pos[element][0]]
            yList += [pos[element][1]]
        xList += [np.nan]
        yList += [np.nan]
    ax.plot(xList, yList, color = colorEdges, alpha= alphaEdges, zorder= 1)

# Plot a random graph
def plotrandomgraph(nodes, radius, xlen, ylen, sizeNodes, colorNodes, colorEdges, alphaEdges):
    Network = createnetwork(nodes, radius, xlen, ylen)
    fig, ax = plt.subplots(figsize = [6*xlen, 6*ylen])
    plt.axis('off')
    plotedges(Network['pos'], ax, Network['sets'], colorEdges, alphaEdges)
    plotnodes(Network['pos'], ax, sizeNodes, colorNodes)

In [ ]:
def linknodesKDtree(pos, radius):
    treePos = spatial.KDTree(pos)
    links = treePos.query_ball_tree(other= treePos, r= radius)
    for i in range(len(links)):
        links[i].remove(i)
    return links

def plotedgeslinks(pos, ax, links, colorEdges, alphaEdges):
    xList = []
    yList = []
    for i in range(len(links)):
        for node in links[i]:
            if node > i:
                xList += [pos[i][0]]
                yList += [pos[i][1]]
                xList += [pos[node][0]]
                yList += [pos[node][1]]
                xList += [np.nan]
                yList += [np.nan]
    ax.plot(xList, yList, color = colorEdges, alpha= alphaEdges, zorder= 1)
    
def removelinks(links, removePerc):
    removeN = round(len(links)*removePerc)
    for i in range(removeN):
        randomNode = random.choice(random.choices(links, weights= map(len, links), k= 1))
        randomConnectedNode = random.choice(links[randomNode])
        links[randomNode].remove(randomConnectedNode)
        links[randomConnectedNode].remove(randomNode)
    return links